In [29]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/rufaelmarew/Documents/tau/finger_pose_estimation')
from util import *

In [2]:
data_path = "/Users/rufaelmarew/Documents/tau/finger_pose_estimation/dataset/Leap_data.csv"
data = pd.read_csv(data_path)
data.head()

,Unnamed: 0,time,timestamp,hand_id,hand_type,palm_x,palm_y,palm_z,Thumb_Metacarpal_position_x,Thumb_Metacarpal_position_y,...,Pinky_Intermediate_rotation_y,Pinky_Intermediate_rotation_z,Pinky_Intermediate_rotation_w,Pinky_Distal_position_x,Pinky_Distal_position_y,Pinky_Distal_position_z,Pinky_Distal_rotation_x,Pinky_Distal_rotation_y,Pinky_Distal_rotation_z,Pinky_Distal_rotation_w
0,0,1.703765e+09,188492264659,111,HandType.Right,-79.167992,362.979034,297.202301,-74.767654,312.221863,...,0.234547,0.014942,0.955898,-69.417786,391.061127,265.167694,-0.560701,0.202716,0.112650,0.794878
1,1,1.703765e+09,188492275791,111,HandType.Right,-77.096741,364.525421,296.313171,-74.891823,315.109161,...,0.226701,-0.015991,0.960225,-65.560516,392.783295,265.652161,-0.553133,0.204778,0.082616,0.803296
2,2,1.703765e+09,188492287313,111,HandType.Right,-74.755348,363.625549,296.456757,-72.420914,314.421051,...,0.236143,-0.015866,0.961739,-63.741043,392.962402,267.041534,-0.542491,0.198801,0.091296,0.811078
3,3,1.703765e+09,188492298190,111,HandType.Right,-72.392403,362.183380,296.547302,-70.115112,313.183838,...,0.236710,-0.001543,0.961843,-61.530251,391.550415,268.474457,-0.540883,0.197743,0.103604,0.810931
4,4,1.703765e+09,188498563054,113,HandType.Right,-45.106735,201.079269,-65.869545,-78.147614,221.092148,...,-0.347708,-0.222745,0.785914,1.476566,178.556274,-72.539337,-0.610987,-0.269604,-0.304878,0.679012


In [23]:
data['time'] = pd.to_datetime(data['time'], unit='s')

,Unnamed: 0,time,timestamp,hand_id,hand_type,palm_x,palm_y,palm_z,Thumb_Metacarpal_position_x,Thumb_Metacarpal_position_y,...,Pinky_Intermediate_rotation_y,Pinky_Intermediate_rotation_z,Pinky_Intermediate_rotation_w,Pinky_Distal_position_x,Pinky_Distal_position_y,Pinky_Distal_position_z,Pinky_Distal_rotation_x,Pinky_Distal_rotation_y,Pinky_Distal_rotation_z,Pinky_Distal_rotation_w
0,0,2023-12-28 11:56:56.469052416,188492264659,111,HandType.Right,-79.167992,362.979034,297.202301,-74.767654,312.221863,...,0.234547,0.014942,0.955898,-69.417786,391.061127,265.167694,-0.560701,0.202716,0.112650,0.794878
1,1,2023-12-28 11:56:56.479566080,188492275791,111,HandType.Right,-77.096741,364.525421,296.313171,-74.891823,315.109161,...,0.226701,-0.015991,0.960225,-65.560516,392.783295,265.652161,-0.553133,0.204778,0.082616,0.803296
2,2,2023-12-28 11:56:56.492596992,188492287313,111,HandType.Right,-74.755348,363.625549,296.456757,-72.420914,314.421051,...,0.236143,-0.015866,0.961739,-63.741043,392.962402,267.041534,-0.542491,0.198801,0.091296,0.811078
3,3,2023-12-28 11:56:56.502609408,188492298190,111,HandType.Right,-72.392403,362.183380,296.547302,-70.115112,313.183838,...,0.236710,-0.001543,0.961843,-61.530251,391.550415,268.474457,-0.540883,0.197743,0.103604,0.810931
4,4,2023-12-28 11:57:02.770024192,188498563054,113,HandType.Right,-45.106735,201.079269,-65.869545,-78.147614,221.092148,...,-0.347708,-0.222745,0.785914,1.476566,178.556274,-72.539337,-0.610987,-0.269604,-0.304878,0.679012
5,5,2023-12-28 11:57:02.780360960,188498573980,113,HandType.Right,-43.078552,198.785553,-63.929707,-77.263031,216.937042,...,-0.411667,-0.208711,0.801055,7.621948,178.712677,-70.120903,-0.535604,-0.340415,-0.298952,0.712653
6,6,2023-12-28 11:57:02.792375040,188498585161,113,HandType.Right,-42.420631,196.995407,-62.438412,-77.080544,214.486603,...,-0.429556,-0.187119,0.813437,10.265207,178.892700,-69.218666,-0.482580,-0.374832,-0.267410,0.745057
7,7,2023-12-28 11:57:02.802556416,188498596293,113,HandType.Right,-41.807121,195.373642,-60.745419,-76.731293,212.558426,...,-0.451757,-0.159410,0.823258,13.148935,179.672913,-68.024101,-0.430271,-0.411391,-0.232831,0.769035
8,8,2023-12-28 11:57:02.814065152,188498607351,113,HandType.Right,-41.216972,193.521423,-59.246536,-76.301834,210.646027,...,-0.466731,-0.135520,0.828476,15.646496,179.907379,-66.909592,-0.395501,-0.430233,-0.206671,0.784707
9,9,2023-12-28 11:57:02.825349120,188498618694,113,HandType.Right,-40.422310,191.650055,-57.964512,-75.668709,208.767639,...,-0.471105,-0.122783,0.831765,17.512217,179.035889,-65.988266,-0.378577,-0.434183,-0.192456,0.794434


In [31]:
emg_path = "/Users/rufaelmarew/Documents/tau/finger_pose_estimation/dataset/leap2_2_BT.edf"
emg_data = read_emg(emg_path)
emg_data.head()

Extracting EDF parameters from /Users/rufaelmarew/Documents/tau/finger_pose_estimation/dataset/leap2_2_BT.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 319749  =      0.000 ...  1278.996 secs...


,EMG Ch-1,EMG Ch-2,EMG Ch-3,EMG Ch-4,EMG Ch-5,EMG Ch-6,EMG Ch-7,EMG Ch-8,EMG Ch-9,EMG Ch-10,EMG Ch-11,EMG Ch-12,EMG Ch-13,EMG Ch-14,EMG Ch-15,EMG Ch-16
time,,,,,,,,,,,,,,,,
2023-10-02 14:59:55.631,54.150622,76.039278,68.743059,58.374749,51.846553,44.550335,65.286956,73.351198,61.062829,43.398300,53.766611,56.070680,36.486093,38.790162,30.341909,-13.051391
2023-10-02 14:59:55.639,12.677380,21.893656,3.845115,15.365460,9.221276,32.645978,25.733771,21.125633,6.533196,45.318358,49.542484,11.525345,52.998588,32.261966,38.022139,64.134921
2023-10-02 14:59:55.647,21.893656,39.942197,14.213426,8.837265,22.661679,32.261966,26.117782,0.773023,32.261966,29.957897,20.741621,-4.219127,8.837265,7.685230,10.373311,13.829414
2023-10-02 14:59:55.655,62.982887,84.103520,65.670967,52.614576,63.366898,64.134921,72.199163,70.279106,54.918645,67.207013,57.222714,66.823002,52.614576,51.846553,56.454691,24.965748
2023-10-02 14:59:55.663,-23.803713,-9.979299,-36.092082,-35.708070,-11.131334,4.613138,-9.595288,-21.883656,-7.291219,21.893656,13.829414,-38.780162,-8.059242,-11.899357,-10.747322,35.718070
